## Chapter 5: Resampling Methods - Lab

In [38]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly,
                         sklearn_sm)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import (train_test_split,
                                     cross_validate,
                                     KFold,
                                     ShuffleSplit)
from sklearn.base import clone

### 5.3.1 Validation Set Approach

We explore the use of the validation set approach in order to estimate the test error rates that result from fitting various linear models on the `Auto` data set.

We use the function `train_test_split()` to split the data into training and validation sets. As there are 392 observations, we split into two equal sets of size 196 using the argument test_size=196.

In [6]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

In [12]:
# Fit a linear regression using the training set
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [18]:
# Perform predictions on validation set
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)

# Calculate the validation MSE
np.mean((y_valid - valid_pred)**2)

23.61661706966988

We can also estimate the validation error for higher-degree polynomial regressions. We frst provide a function `evalMSE()` that takes a model string as well as a training and test set and returns the MSE on the test set.

In [21]:
def evalMSE(terms, response, train, test):

    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((y_test - test_pred)**2)

In [22]:
# Estimate the validation MSE for linear, quatratic and cubic fits
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE(
        [poly('horsepower', degree)],
        'mpg',
        Auto_train,
        Auto_valid
    )

MSE

array([23.61661707, 18.76303135, 18.79694163])

These results are consistent with our previous fndings: a model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower, and there is no evidence of an improvement in using a cubic function of horsepower.

The `ISLP` package contains a wrapper, `sklearn_sm()`, which lets us easily use the cross-validation tools of `sklearn` with models fit by `statsmodels`.

The class `sklearn_sm()` has as its frst argument a model from statsmodels. It can take two additional optional arguments: `model_str` which can be used to specify a formula, and `model_args` which should be a dictionary of additional arguments used when ftting the model. For example, to fit a logistic regression model we have to specify a family argument. This is passed as `model_args={'family':sm.families.Binomial()}`.

In [24]:
hp_model = sklearn_sm(
    sm.OLS,
    MS(['horsepower'])
)

X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

In [27]:
# Try polynomial fits with degree 1 to 5
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d + 1))
    M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.03322411])

In the CV example above, we used `K = n`, but of course we can also use `K < n`. The code is very similar to the above (and is signifcantly faster).

Here we use `KFold()` to partition the data into `K = 10` random groups.

In [39]:
cv_error = np.zeros(5)
cv = KFold(
    n_splits=10,
    shuffle=True,
    random_state=0
) # use same splits for each degree

for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d + 1))
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13719154])

We still see little evidence that using cubic or higher-degree polynomial terms leads to a lower test error than simply using a quadratic ft.

The `cross_validate()` function is fexible and can take different splitting mechanisms as an argument. For instance, one can use the `ShuffleSplit()` function to implement the validation set approach just as easily as K-fold cross-validation.

In [40]:
validation = ShuffleSplit(
    n_splits=1,
    test_size=196,
    random_state=0
)

results = cross_validate(
    hp_model,
    Auto.drop(['mpg'], axis=1),
    Auto['mpg'],
    cv=validation
)

results['test_score']

array([23.61661707])

In [41]:
# estimate the variability in the test error across 10 splits
validation = ShuffleSplit(
    n_splits=10,
    test_size=196,
    random_state=0
)
results = cross_validate(
    hp_model,
    Auto.drop(['mpg'], axis=1),
    Auto['mpg'],
    cv=validation
)

results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091831)

Note that this standard deviation is not a valid estimate of the sampling variability of the mean test score or the individual scores, since the randomly-selected training samples overlap and hence introduce correlations. But it does give an idea of the Monte Carlo variation incurred by picking diferent random folds.

### 5.3.3 The Bootstrap

Estimating the acuracy of a statistic of interest

To illustrate the bootstrap, we start with a simple example using the `Portfolio` dataset in the ISLP package. The goal is to estimate the sampling variance of the parameter `α`, the fraction of money invested in `X` (with the rest invested in `Y`).

We will create a function `alpha_func()`, which takes as input a dataframe `D` assumed to have columns `X` and `Y`, as well as a vector `idx` indicating which observations should be used to estimate `α`. The function then outputs the estimate for `α` based on the selected observations.

In [51]:
# Return an estimate for alpha which minimises risk
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [53]:
Portfolio = load_data('portfolio')

In [54]:
# Estimate alpha which minimises risk using all 100 observations
alpha_func(Portfolio, range(100))


0.57583207459283

Next we randomly select 100 observations from `range(100)`, with replacement. This is equivalent to constructing a new bootstrap data set.

In [57]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619004

This process can be generalized to create a simple function `boot_SE()` for computing the bootstrap standard error for arbitrary functions that take only a data frame as an argument.

In [60]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    
    return np.sqrt(second_ / B - (first_ / B)**2)

In [62]:
# Estimate the accuracy of alpha using 1000 bootstrap replicants
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)

alpha_SE

0.09118176521277699

#### Estimating the accuracy of a linear regression model

We start by writing a generic function `boot_OLS()` for bootstrapping a regression model that takes a formula to defne the corresponding regression. We use the `clone()` function to make a copy of the formula that can be refit to the new dataframe. This means that any derived features such as those defined by `poly()` (which we will see shortly), will be re-fit on the resampled data frame.

In [63]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [64]:
 hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [65]:
# Create 10 bootstrap estimate for the intercept and slope terms
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)])

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [66]:
# Compute the standard errors of 1000 boostrap estimates for the intercept and slope terms
hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)

hp_se

intercept     0.848807
horsepower    0.007352
dtype: float64

The standard errors for the regression coefficients in a linear model can also be ogtained using the `summary()` method.

In [70]:
hp_model.fit(Auto, Auto['mpg'])
model_se = hp_model.results_.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Sat, 26 Aug 2023   Prob (F-statistic):           7.03e-81
Time:                        14:04:58   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145
==============================================================================
Omnibus:                       16.432   Durbin-Watson:                   0.920
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.305
Skew:                           0.492   Prob(JB):                     0.000175
Kurtosis:                       3.299   Cond. No.                         322.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [71]:
# Compute the bootstrap standard error estimates from fitting
# a quadratic model to the data
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(
    boot_OLS,
    quad_model,
    'mpg'
)

boot_SE(quad_func, Auto, B=1000)

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [72]:
# Show the standard errors computes using sm.OLS()
M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))

M.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     428.0
Date:                Sat, 26 Aug 2023   Prob (F-statistic):           5.40e-99
Time:                        14:14:13   Log-Likelihood:                -1133.2
No. Observations:                 392   AIC:                             2272.
Df Residuals:                     389   BIC:                             2284.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
intercept                                  56.9001      1.800     31.604      0.000      53.360      60.440
poly(horsepower, degree=2, raw=True)[0]    -0.4662      0.031    -14.978      0.000      -0.527      -0.405
poly(horsepower, degree=2, raw=True)[1]     0.0012      0.000     10.080      0.000       0.001       0.001
==============================================================================
Omnibus:                       16.158   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.662
Skew:                           0.218   Prob(JB):                     2.20e-07
Kurtosis:                       4.299   Cond. No.                     1.29e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Because the quadratic model provides a much better fit to the data, there's now better agreement between the bootstap estimates of the standard errors and the standard estimates.